In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes
from time import time

from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RandomizedSearchCV, ShuffleSplit

properati = pd.read_csv('datos/caba_para_mapa.csv',error_bad_lines=False)

#tenia un nan nose porque
properati.dropna(inplace=True)

<IPython.core.display.Javascript object>

In [5]:
cant_buckets = 15

#buckets
cantidad,rango = np.histogram(properati['price_usd_per_m2'], bins=cant_buckets)
properati['categories_by_price']=pd.cut(properati['price_usd_per_m2'],rango,labels=np.arange(cant_buckets))
properati['price_range']=pd.cut(properati['price_usd_per_m2'],rango)
#lo casteo a float porque si no tira error 
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64) 

#tenia un nan nose porque
properati.dropna(inplace=True)

In [6]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['categories_by_price']

#reviso si quedo datos anomalos
print (np.any(np.isnan(y)) == False)
print (np.all(np.isfinite(y)) == True )

True
True


### Busco hiper-parametros con random search

In [14]:
%%notify

#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['categories_by_price']

gtb = GradientBoostingClassifier()

n_estimators = np.arange(100,200,5)
criterion = ["friedman_mse","mse","mae"]
max_features =["auto","sqrt","log2",None]

param_grid = {"n_estimators": n_estimators,"criterion":criterion,"max_features":max_features}
custom_cv = ShuffleSplit(n_splits=10, test_size=0.2)

random_search = RandomizedSearchCV(gtb, param_distributions=param_grid ,cv=custom_cv,n_iter=11,n_jobs=-1)
start = time()
random_search.fit(X, y)

print("RandomizedSearchCV duro %.2f segundos para %d candidatos a hyper-parametros."
    % (time() - start, len(random_search.cv_results_['params'])))
print("")
score.report_single(random_search.cv_results_)

KeyboardInterrupt: 

<IPython.core.display.Javascript object>

KeyboardInterrupt: 